# Project 1 - Tile image pre-processing (v1)

In Project 1, your `tile-images` directory is used for your tiles to make your mosaic.  This program will pre-process your tile images to do the following:

1. Convert every image to an RGB format (in case they are greyscale or transparent),
  - Grayscale and transparent images don't have full color data and you can't find the average color without color data! :(
2. Scale every image to a smaller size, and
  - Smaller images will be faster to calculate the average image color.
3. Transform your image to a square image
  - The final mosaic only uses square regions of your image -- by pre-processing your images into squares you will get the best possible average color.
4. Delete any non-image files
  - Mac OSX likes to put a file called `.DS_Stroe` in with your images.  This can cause some problems for Mac users.
  
There are few options you can configure then run this pre-processing! :)

## Options

Adjust the following options if needed (or just run the cell with the defaults, which are good for most people):

In [ ]:
# What is the `width` of your final image?  (int, as the number of pixels; default: 100)
# - Smaller image == less quality, faster speed
tileImageWidth = 100

# Force re-saving of images (True/False, default: False)
# - This program will not re-process and re-save images that are already squares of the right shape.
# - If you find that you have iamges that are still a problem after running this script, change this to True.
forceResave = False

## Image Converter

In [ ]:
import PIL
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
import sys

# What directory includes the images?
imageDir = "tile-images"

# Finds all files inside of `imageDir`:
files = [imageDir + "/" + f for f in listdir(imageDir + "/") if isfile(join(imageDir + "/", f))]

# Loop through every file:
ct = 0
print(f"Starting to process {len(files)} files...")
for file in files:
    ## == Read File ==
    try:
        im = Image.open(file)
        w, h = im.size
        d = min(w, h)
        scale = tileImageWidth / d

    except:
        sys.stderr.write(f"{file} is not an image. Removing file.\n")
        os.remove(file)
        continue
        
    imageChanged = False


    ## == Crop Image ==
    
    # If the image is not a square, make it a square:
    if w != h:
        # If the width is smaller than the height, crop the height:
        if d == w:
            # Final height is the scale of the height:
            final_h = w
            start_h = (h - final_h) / 2

            final_w = w
            start_w = 0

        # Otherwise, crop the width:
        else:
            final_w = h
            start_w = (w - final_w) / 2

            final_h = h
            start_h = 0

        z = (start_w, start_h, final_w, final_h)
        im = im.crop( (start_w, start_h, final_w, final_h) )
        imageChanged = True
    
    
    ## == Resize Image ==
    
    # If the image is smaller than the given size (or close to it), don't resize it at all.
    #   (Resizing combines pixels, so resizing by only a few pixels will be very lossy.)
    if scale < 0.5:
        im = im.resize( (tileImageWidth, tileImageWidth), resample=PIL.Image.LANCZOS )
        imageChanged = True
    
    
    ## == Save Image ==
    if imageChanged or forceResave:
        rgbImage = Image.new('RGB', (tileImageWidth, tileImageWidth))
        rgbImage.paste(im, (0, 0))

        # Ensure we save it as a JPG file:
        newFileName = file
        if file.endswith(".png"):
            newFileName = file.replace(".png", ".jpg")
        elif file.endswith(".PNG"):
            newFileName = file.replace(".PNG", ".jpg")
        elif file.endswith(".gif"):
            newFileName = file.replace(".gif", ".jpg")
        elif file.endswith(".bmp"):
            newFileName = file.replace(".bmp", ".jpg")

        # Save the file:
        rgbImage.save(newFileName)

        # If we renamed the file, delete the old one:
        if newFileName != file:
            os.remove(file)       
    
    ct = ct + 1
    pct = ct / len(files) * 100
    sys.stdout.write(f"\r  ... {ct} / {len(files)} total files processed ({pct:.2f}%)")
    
tada = "\N{PARTY POPPER}"
print("")
print("")
print(f"{tada} Image pre-processing complete.")